# LangChain
LangChain is a framework for developing applications powered by language models.

## Schema
Most interfaces in LangChain are text based because most language models support only "text in, text out" (if you are interested, we can have an hour long discussion where I can explain you everything about transformers architecture).

### How LLMs work (practical approach, not theory)
LLMs have 3 parts in prompt usually.
1. System Chain Message
    - A chat message representing information that should be instructions to the AI system.
2. Human Chat Message
    - A chat message representing information coming from a human interacting with the AI system.
3. AI Chat Message
    - A chat message representing information coming from the AI system.

**__PRO_TIP__**: While making product, try to keep Human chat as low as possible because normal users are very bad at writing prompt and model give pretty bad results with bad prompts which will results into you replying to useless complaint emails.

Other than that, one important thing I need to explain.
>    - Our daily usage is when we use model with zero-shot learning but model give outstanding result when it is fine-tuned with some examples, usually known as few-shot learning. We use method of `Examples` for this where we provide model with input/output pairs so model can better understand what we are expecting. 

While talking about terminology, we use word `Document` for unstructured data. Documents have compeletely random formats so normal programming approach cannot get desired data from doc. That is why we use LLMs because they can really understand data and give response accordingly. Document has two parts, `page_content` and `metadata`.